In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
def GetPlottingDirectory(plotFileName, plotType):
    plottingDirectory = mainCodeDirectory=os.path.join(mainDirectory,"Code","PLOTTING")
    
    specificPlottingDirectory = os.path.join(plottingDirectory, plotType, 
                                             f"{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz")
    os.makedirs(specificPlottingDirectory, exist_ok=True)

    plottingFileName=os.path.join(specificPlottingDirectory, plotFileName)

    return plottingFileName

def SaveFigure(fig,plotType, fileName):
    plotFileName = f"{fileName}_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}.jpg"
    plottingFileName = GetPlottingDirectory(plotFileName, plotType)
    print(f"Saving figure to {plottingFileName}")
    fig.savefig(plottingFileName, dpi=300, bbox_inches='tight')

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","1_Domain_Profiles"))
from CLASSES_DomainProfiles import DomainProfiles_Class, DomainProfiles_DataLoading_Class

In [ ]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
dataName="UpdraftArea"

#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Domain_Profiles", dataName=dataName,
                                dtype='float32',codeSection = "Project_Algorithms")

In [ ]:
####################################
#PLOTTING FUNCTIONS

In [ ]:
def LoadMeanLFC():
    # dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    # in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    # with open(in_file, 'rb') as f:
    #     MeanLFC = pickle.load(f)
    MeanLFC=2 #*#*
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC}\n")


def LoadAllCloudBase():
    # dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    # in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    # with open(in_file, 'rb') as f:
    #     all_cloudbase = pickle.load(f)
    all_cloudbase=1 #*#*
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {cloudbase}\n")

In [ ]:
#thresholds
w_thresh1,w_thresh2,qcqi_thresh = DomainProfiles_DataLoading_Class.GetCloudyUpdraftThresholds()

In [ ]:
def LoadProfiles(datatype, masked, varNames):
    Dictionary = DomainProfiles_DataLoading_Class.LoadProfile(ModelData,DataManager, dataName, datatype,"timeaverage",masked)

    Dictionary_SE = {}
    for varName in varNames:
        one=Dictionary[f"{varName}_{datatype}_timeaverage"]
        two=Dictionary[f"{varName}_squares_{datatype}_timeaverage"]
        Dictionary_SE[f"{varName}_{datatype}_timeaverage_SE"] = ProfileStandardError(one,two)

    factor=1.96
    return Dictionary, Dictionary_SE, factor

In [ ]:
def GetMultiplyFactor(varName):
    if varName in ["UpdraftArea_g","UpdraftArea_c"]:
        multiplyFactor = 1/(1e3**2)

    else:
        multiplyFactor = 1
    return multiplyFactor

def PlotProfile(profile, axis, label, color, xlabel, linestyle='solid'):
    axis.plot(profile[:, 0], profile[:, 1], label=label, color=color, linestyle=linestyle)
    axis.set_xlabel(xlabel)
    axis.set_ylabel("z (km)")
    axis.grid(True)
    
def PlotProfile_SE(profileMean,profile_SE,color,factor, axis):
    axis.fill_betweenx(profileMean[:, 1],
                       profileMean[:, 0] - factor*profile_SE[:, 0],
                       profileMean[:, 0] + factor*profile_SE[:, 0],
                       color=color, alpha=0.1)

def PlotProfiles(varNames, maskeds, datatypes, xlabels, title, x_scale=1, y_scale=1):
    """
    Plot vertical profiles for each variable in varNames.
    Each variable gets its own subplot, with lines for each datatype (e.g. 'general', 'cloudy').
    """
    n_varNames = len(varNames)
    n_cols = int(np.ceil(np.sqrt(n_varNames)))
    n_rows = int(np.ceil(n_varNames / n_cols))

    fig = plt.figure(figsize=(5 * n_cols * y_scale, 4 * n_rows * x_scale))
    
    gs = gridspec.GridSpec(n_rows, n_cols, figure=fig)

    # consistent color scheme for datatypes
    color_map = {'all': 'black', 'general': 'green', 'cloudy': 'blue'}

    for count, (varName, datatype,masked, xlabel) in enumerate(zip(varNames,datatypes,maskeds, xlabels)):
        row, col = divmod(count, n_cols)
        ax = fig.add_subplot(gs[row, col])

    
        Dictionary, Dictionary_SE, factor = LoadProfiles(datatype, masked, [varName])

        profile = Dictionary[f"{varName}_{datatype}_timeaverage"]
        profileMean = ProfileMean(profile)

        profile_SE = Dictionary_SE[f"{varName}_{datatype}_timeaverage_SE"]

        color = color_map.get(datatype, 'gray')
        label = f"{datatype}"
        if datatype == 'all':
            label = f"domain"

        multiplyFactor = GetMultiplyFactor(varName)
        profileMean[:,0]*=multiplyFactor
        profile_SE[:,0]*=multiplyFactor
        
        PlotProfile(profileMean, ax, label=label, color=color, xlabel=xlabel)
        PlotProfile_SE(profileMean,profile_SE, color, factor, ax)

        ax.set_ylim(0,20)
        ax.legend()
        ax.grid(True)


    axs = fig.get_axes()
    #FIXING TICKS
    #left xlimit
    for axis in axs:
        axis.set_xlim(left=0)

    #matching UpdraftArea_g and UpdraftArea_c xrange
    ax_lst=[axs[0],axs[1]]; MatchAxisLimits(ax_lst,dim='x')

    #snapping limits
    SnapLimitsToTicks(axs, dim='x')
    
    #ADDING AXLINES
    axline_lw=1.1
    for axis in axs:
        axis.axhline(cloudbase, color='purple', linestyle='dashed', lw=axline_lw)
        axis.axhline(MeanLFC, color='green', linestyle='dashed', lw=axline_lw)

    #FIXING FIGURE LAYOUT
    # fig.tight_layout() #incompatible with SnapLimitsToTIcks
    # fig.subplots_adjust(top=0.95, wspace=0.2, hspace=0.2)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    #setting figure title
    fig.suptitle(title)

    return fig

In [ ]:
####################################
#RUNNING

In [ ]:
datatypes = ['general','cloudy']
maskeds = [True,True]
varNames=['UpdraftArea_g','UpdraftArea_c'] 
xlabels = [
    r'$(km)^2$', 
    r'$(km)^2$'
]

title=f"Average General vs Cloudy UpdraftArea Profiles"
fig = PlotProfiles(varNames, maskeds, datatypes, xlabels, title, x_scale=1.3, y_scale=1.3)

fileName=f"Eulerian_Profiles_UpdraftArea" 
SaveFigure(fig,plotType="Project_Algorithms/Domain_Profiles/Eulerian_Profiles_UpdraftArea",fileName=fileName)